In [1]:
from MultivariateNormalDistribution import mvn

import numpy as np

In [5]:
print("======Creating Ground Truth=====\n")
dimensions = 3
mu = (np.random.uniform(0,1,dimensions)).reshape(dimensions,1)*20
print("Ground truth mu: \n{}".format(mu))
#covariance = datasets.make_spd_matrix(dimensions) #np.random.rand(dimensions,dimensions) # TODO gallery??
covariance = np.identity(dimensions)

MVN1 = mvn.MultivariateNormalDistribution(dimensions=dimensions, mean=mu, covariance=covariance, name="MVN1")

# Sample MVN1
N = 5
samples = MVN1.sampleDistribution(N)

print("\n======Setting parameters for MAP=====\n")

mvn_prior = mvn.MultivariateNormalDistribution(dimensions=dimensions, name="mvn_prior")
mvn_prior.MLE(samples)

nu0 = N;
S_0 = np.multiply(np.diag(np.diag(mvn_prior.sigma)),nu0)
m_0 = mvn_prior.mu
kappa0 = N

print("\n======Creating MVN2(using MLE) and MVN3(using MAP)=====\n")

# Sample MVN1 again
N = 50
samples = MVN1.sampleDistribution(N)
# MVM2 will only do MLE
MVN2 = mvn.MultivariateNormalDistribution(dimensions=dimensions, name="MVN2");
# MVN3 will have a prior and do posterior analysis
MVN3 = mvn.MultivariateNormalDistribution(dimensions=dimensions, m0=m_0, S0=S_0 , kappa=kappa0, nu=nu0, name="MVN3")

MVN2.MLE(samples)
MVN3.MAP(samples)


print("\n=====Compute likelihood========\n")
N = 5
samples_test = MVN1.sampleDistribution(N)

lik1 = MVN1.logpdf(samples_test)
lik2 = MVN2.logpdf(samples_test)
lik3 = MVN3.logpdf(samples_test)
loglik1 = MVN1.loglikelihood(samples_test)
loglik2 = MVN2.loglikelihood(samples_test)
loglik3 = MVN3.loglikelihood(samples_test)
lik3_2 = MVN3.logPosteriorPredictive(samples_test)

print("likelihood of {}: \n{}".format(MVN1.name,lik1))
print("loglikelihood of {}: \n{}\n".format(MVN1.name,loglik1))
print("likelihood of {}: \n{}".format(MVN2.name,lik2))
print("loglikelihood of {}: \n{}\n".format(MVN2.name,loglik2))
print("likelihood of {}: \n{}".format(MVN3.name,lik3))
print("loglikelihood of {}: \n{}".format(MVN3.name,loglik3))
print("likelihod tstudent of {}: \n{}".format(MVN3.name, lik3_2))

======Creating Ground Truth=====

Ground truth mu: 
[[12.07244349]
 [12.35270012]
 [15.38369646]]

======Setting parameters for MAP=====


======Creating MVN2(using MLE) and MVN3(using MAP)=====


=====Compute likelihood========

logPosteriorPredictive dof: 53
logPosteriorPredictive kappa0: 55
likelihood of MVN1: 
[[-5.52023764]
 [-3.40838139]
 [-3.10115443]
 [-3.80721581]
 [-3.85289909]]
loglikelihood of MVN1: 
-19.68988835925526

likelihood of MVN2: 
[[-6.3012323 ]
 [-3.4255439 ]
 [-3.23809421]
 [-3.4780188 ]
 [-3.92177314]]
loglikelihood of MVN2: 
-20.36466236037602

likelihood of MVN3: 
[[-6.44521612]
 [-3.39335791]
 [-3.14586715]
 [-3.42170805]
 [-3.83715972]]
loglikelihood of MVN3: 
-20.243308949086185
likelihod tstudent of MVN3: 
None
